In [1]:
import pandas as pd
import numpy as np
from functions import *
import random
import csv
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
#Load models
#REMINDER: We are going to need to use 'eval' to get the models usable
with open('models.csv', newline='') as f:
    reader = csv.reader(f)
    models = list(reader)
    
print(len(models))


###SELECT MODEL TO OPTIMIZE
model_num = 4

5


In [4]:
models

[['model_1', 'model_2', 'model_3', 'model_4', 'model_5', ''],
 ["LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,\n                   intercept_scaling=1, l1_ratio=None, max_iter=50000,\n                   multi_class='auto', n_jobs=None, penalty='l2',\n                   random_state=75, solver='lbfgs', tol=5.993308915320019,\n                   verbose=0, warm_start=False)",
  "DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',\n                       max_depth=None, max_features=None, max_leaf_nodes=262,\n                       min_impurity_decrease=0.0, min_impurity_split=None,\n                       min_samples_leaf=1, min_samples_split=2,\n                       min_weight_fraction_leaf=0.0, presort='deprecated',\n                       random_state=75, splitter='best')",
  "RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,\n                       criterion='gini', max_depth=None, max_features='auto',\n  

In [5]:
df = pd.read_csv("../data/ufc-master.csv")

In [6]:
#Let's put all the labels in a dataframe
df['label'] = ''
#If the winner is not Red or Blue we can remove it.
mask = df['Winner'] == 'Red'
df['label'][mask] = 0
mask = df['Winner'] == 'Blue'
df['label'][mask] = 1

#df["Winner"] = df["Winner"].astype('category')
df = df[(df['Winner'] == 'Blue') | (df['Winner'] == 'Red')]


#Make sure lable is numeric
df['label'] = pd.to_numeric(df['label'], errors='coerce')

C:\Users\Matt-Predator\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\Matt-Predator\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [7]:
#Let's fix the date
df['date'] = pd.to_datetime(df['date'])

In [8]:
#Create a label df:
label_df = df['label']

#Let's create an odds df too:
odds_df = df[['R_odds', 'B_odds']]

In [9]:
#Split the test set.  We are always(?) going to use the last 200 matches as the test set, so we don't want those around
#as we pick models

df_train = df[200:]
odds_train = odds_df[200:]
label_train = label_df[200:]

df_test = df[:200]
odds_test = odds_df[:200]
label_test = label_df[:200]

print(len(df_test))
print(len(odds_test))
print(len(label_test))

print(len(df_train))
print(len(odds_train))
print(len(label_train))

200
200
200
4023
4023
4023


In [10]:
display(df_train)
display(df_test)

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,sub_dif,height_dif,reach_dif,age_dif,sig_str_dif,avg_sub_att_dif,avg_td_dif,empty_arena,constant_1,label
201,Mirsad Bektic,Dan Ige,-125.0,105.0,80.000000,105.00,2020-02-08,"Houston, Texas, USA",USA,Blue,...,0.0,-2.54,2.54,0.0,1.270000,0.600000,-0.650000,0.0,1.0,1
202,Derrick Lewis,Ilir Latifi,-225.0,185.0,44.444444,185.00,2020-02-08,"Houston, Texas, USA",USA,Red,...,2.0,-12.70,-15.24,-1.0,-0.340000,0.500000,1.330000,0.0,1.0,0
203,Trevin Giles,James Krause,-170.0,130.0,58.823529,130.00,2020-02-08,"Houston, Texas, USA",USA,Red,...,1.0,5.08,-2.54,-6.0,1.400000,0.600000,-0.890000,0.0,1.0,0
204,Alex Morono,Kalinn Williams,-345.0,285.0,28.985507,285.00,2020-02-08,"Houston, Texas, USA",USA,Blue,...,-1.0,2.54,0.00,4.0,-4.420000,-0.600000,0.000000,0.0,1.0,1
205,Lauren Murphy,Andrea Lee,260.0,-320.0,260.000000,31.25,2020-02-08,"Houston, Texas, USA",USA,Red,...,0.0,2.54,5.08,6.0,1.490000,0.500000,0.860000,0.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4291,Duane Ludwig,Darren Elkins,-155.0,135.0,64.516129,135.00,2010-03-21,"Broomfield, Colorado, USA",USA,Blue,...,0.0,0.00,2.54,6.0,-13.666667,0.000000,0.000000,0.0,1.0,1
4292,John Howard,Daniel Roberts,-210.0,175.0,47.619048,175.00,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,0.0,7.62,7.62,-2.0,-18.000000,-1.000000,-4.666667,0.0,1.0,0
4293,Brendan Schaub,Chase Gormley,-260.0,220.0,38.461538,220.00,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,0.0,-2.54,-2.12,0.0,-4.000000,1.000000,1.000000,0.0,1.0,0
4294,Mike Pierce,Julio Paulino,-420.0,335.0,23.809524,335.00,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,0.0,10.16,7.62,-5.0,-40.500000,0.000000,-3.500000,0.0,1.0,0


,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,sub_dif,height_dif,reach_dif,age_dif,sig_str_dif,avg_sub_att_dif,avg_td_dif,empty_arena,constant_1,label
0,Amanda Nunes,Felicia Spencer,-600.0,450.0,16.666667,450.000000,2020-06-06,"Las Vegas, Nevada, USA",USA,Red,...,-1.0,-5.08,-2.54,-3.0,0.20,0.0,-0.97,1.0,1.0,0
1,Raphael Assuncao,Cody Garbrandt,120.0,-140.0,120.000000,71.428571,2020-06-06,"Las Vegas, Nevada, USA",USA,Blue,...,-1.0,7.62,-2.54,-9.0,0.21,-0.3,-0.60,1.0,1.0,1
2,Aljamain Sterling,Cory Sandhagen,-115.0,-105.0,86.956522,95.238095,2020-06-06,"Las Vegas, Nevada, USA",USA,Red,...,-2.0,10.16,-2.54,-2.0,2.29,-0.2,-0.67,1.0,1.0,0
3,Neil Magny,Anthony Rocco Martin,-135.0,115.0,74.074074,115.000000,2020-06-06,"Las Vegas, Nevada, USA",USA,Red,...,2.0,-7.62,-17.78,-2.0,-1.23,0.8,-2.02,1.0,1.0,0
4,Eddie Wineland,Sean O'Malley,400.0,-500.0,400.000000,20.000000,2020-06-06,"Las Vegas, Nevada, USA",USA,Blue,...,0.0,10.16,7.62,-10.0,3.46,1.1,0.96,1.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,Claudio Silva,Cole Williams,-550.0,425.0,18.181818,425.000000,2019-08-03,"Newark, New Jersey, USA",USA,Red,...,-2.0,2.54,5.08,1.0,-2.03,-2.1,-3.32,0.0,1.0,0
197,Miranda Granger,Hannah Goldy,-105.0,-115.0,95.238095,86.956522,2019-08-03,"Newark, New Jersey, USA",USA,Red,...,0.0,-7.62,-17.78,0.0,3.06,0.0,0.00,0.0,1.0,0
198,Jon Jones,Dominick Reyes,-460.0,365.0,21.739130,365.000000,2020-02-08,"Houston, Texas, USA",USA,Red,...,-4.0,0.00,-17.78,2.0,0.72,-0.1,-1.51,0.0,1.0,0
199,Valentina Shevchenko,Katlyn Chookagian,-920.0,610.0,10.869565,610.000000,2020-02-08,"Houston, Texas, USA",USA,Red,...,-2.0,10.16,5.08,0.0,1.33,-0.3,-1.94,0.0,1.0,0


In [11]:
#1. Set features
#2. Set Hyperparameters
#3. Set EV
#4. Remove Features

In [12]:
models[1][model_num]

'GaussianNB(priors=None, var_smoothing=1e-12)'

In [13]:
test_model_name = models[0][model_num]
test_model = eval(models[1][model_num])
test_model_features = eval(models[2][model_num])
test_model_ev = eval(models[3][model_num])

In [14]:
old_test_model = test_model
old_test_model_features = test_model_features
old_test_model_ev = test_model_ev

In [15]:
#1. set features
my_pos_features = ['R_fighter', 'B_fighter', 'R_odds', 'B_odds', 'R_ev', 'B_ev',
       'location', 'country', 'title_bout', 'weight_class', 'gender',
       'no_of_rounds', 'B_current_lose_streak', 'B_current_win_streak',
       'B_draw', 'B_avg_SIG_STR_landed', 'B_avg_SIG_STR_pct', 'B_avg_SUB_ATT',
       'B_avg_TD_landed', 'B_avg_TD_pct', 'B_longest_win_streak', 'B_losses',
       'B_total_rounds_fought', 'B_total_title_bouts',
       'B_win_by_Decision_Majority', 'B_win_by_Decision_Split',
       'B_win_by_Decision_Unanimous', 'B_win_by_KO/TKO', 'B_win_by_Submission',
       'B_win_by_TKO_Doctor_Stoppage', 'B_wins', 'B_Stance', 'B_Height_cms',
       'B_Reach_cms', 'B_Weight_lbs', 'R_current_lose_streak',
       'R_current_win_streak', 'R_draw', 'R_avg_SIG_STR_landed',
       'R_avg_SIG_STR_pct', 'R_avg_SUB_ATT', 'R_avg_TD_landed', 'R_avg_TD_pct',
       'R_longest_win_streak', 'R_losses', 'R_total_rounds_fought',
       'R_total_title_bouts', 'R_win_by_Decision_Majority',
       'R_win_by_Decision_Split', 'R_win_by_Decision_Unanimous',
       'R_win_by_KO/TKO', 'R_win_by_Submission',
       'R_win_by_TKO_Doctor_Stoppage', 'R_wins', 'R_Stance', 'R_Height_cms',
       'R_Reach_cms', 'R_Weight_lbs', 'R_age', 'B_age', 'lose_streak_dif',
       'win_streak_dif', 'longest_win_streak_dif', 'win_dif', 'loss_dif',
       'total_round_dif', 'total_title_bout_dif', 'ko_dif', 'sub_dif',
       'height_dif', 'reach_dif', 'age_dif', 'sig_str_dif', 'avg_sub_att_dif',
       'avg_td_dif', 'empty_arena']

In [16]:
print(test_model_name)
print(test_model)
print(test_model_features)
print(test_model_ev)



model_5
GaussianNB(priors=None, var_smoothing=1e-12)
['height_dif', 'sub_dif', 'B_draw', 'avg_td_dif', 'R_win_by_Decision_Split', 'age_dif', 'B_current_win_streak', 'R_odds']
0


In [ ]:
keep_going = True
while(keep_going):
    #1. Set Features
    #get_best_features(pos_features, m, df, cur_features, labels, odds, scale=False)
    test_model_features = (get_best_features(my_pos_features, test_model, df_train, test_model_features, label_train, odds_train, 
                                             min_ev=test_model_ev))

    #2 Set hyperparameters
    #def tune_hyperparameters(input_model, input_features, input_df, input_labels, odds_input):
    test_model = tune_hyperparameters(test_model, test_model_features, df_train, label_train, odds_train, 
                                      min_ev=test_model_ev)

    #3. Set EV
    #def tune_ev(input_model, input_features, input_df, input_labels, odds_input, verbose=False):
    test_model_ev = tune_ev(test_model, test_model_features, df_train, label_train, odds_train, verbose=False)

    #4. Remove Features
    #def remove_to_improve(cur_features, m, df, labels, odds, scale=False, min_ev = 0):
    test_model_features = remove_to_improve(test_model_features, test_model, df_train, label_train, odds_train, min_ev = test_model_ev)    
    keep_going = False
    if old_test_model != test_model:
        print("The hyperparameters are different")
        print("OLD:")
        print(old_test_model)
        print("NEW:")
        print(test_model)
        keep_going = True
        old_test_model = test_model
    if old_test_model_features != test_model_features:
        print("The features are different")
        print("OLD:")
        print(old_test_model_features)
        print("NEW:")
        print(test_model_features)
        keep_going = True
        old_test_model_features = test_model_features
    if old_test_model_ev != test_model_ev:
        print("The EV is different")
        print("OLD:")
        print(old_test_model_ev)
        print("NEW:")
        print(test_model_ev)
        keep_going = True
        old_test_model_ev = test_model_ev
        
    

Current best score is: 38.57057552701814
NO IMPROVEMENT
FINAL BEST SCORE: 38.57057552701814


Starting New Run for GaussianNB


Previous Best Score: 38.57057552701814
var_smoothing: 1e-12 Score: 38.57057552701814
var_smoothing: 1e-11 Score: 38.57057552701814
var_smoothing: 1e-10 Score: 38.33510081217463
var_smoothing: 1e-09 Score: 37.509082183713524
var_smoothing: 1e-08 Score: 34.49717738669538
var_smoothing: 1e-07 Score: 27.533575812587266
var_smoothing: 1e-06 Score: 19.268778405048693
0 28795.247500911966
1 26884.099375518985
2 26301.469837850807
3 24609.421573922482
4 23804.882260983904
5 22688.526718697864
6 21949.043290543446
7 21235.95727318451
8 20737.822868588533
9 20967.49782311879
10 20636.22154159069
11 21136.75463579103
12 21010.931270211175
13 20169.514895052613
14 18595.278123327113
15 18102.891563776426
16 17559.1015923189
17 18070.404967836672
18 17495.442603110485
19 16915.697924353757
20 15460.892936359547
21 14318.966634904291
22 14350.778814354468
23 13753.732697184

In [ ]:
#Get new score:
#def evaluate_model(input_model, input_features, input_ev, train_df, train_labels, train_odds, test_df, test_labels, test_odds, verbose=True):
score = evaluate_model(test_model, test_model_features, test_model_ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)

print(test_model_name) 
print(models[0][model_num])
print()
print(test_model)
print(eval(models[1][model_num]))
print()
print(test_model_features) 
print(eval(models[2][model_num]))
print()
print(test_model_ev)
print(eval(models[3][model_num]))

In [ ]:
models[0][model_num] =  test_model_name 
models[1][model_num] = test_model
models[2][model_num] = test_model_features
models[3][model_num] = test_model_ev
models[4][model_num] = score

In [ ]:
with open('models.csv', 'w', newline='') as outfile:
    writer = csv.writer(outfile)
    for row in models:
        print("HI")
        writer.writerow(row)
    
outfile.close()

In [ ]:
with open('models.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)
    
print(len(data))

In [ ]:
data